In [16]:
#https://www.teamrankings.com/nfl/stat/points-per-game

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

team_list = ["Arizona", "Atlanta", "Baltimore", "Buffalo", "Carolina", "Chicago", "Cincinnati", "Cleveland",
            "Dallas", "Denver", "Detroit", "Green Bay", "Houston", "Indianapolis", "Jacksonville", "Kansas City",
            "LA Chargers", "LA Rams", "Las Vegas", "Miami", "Minnesota", "New England", "New Orleans", "NY Giants",
            "NY Jets", "Philadelphia", "Pittsburgh", "San Francisco", "Seattle", "Tampa Bay", "Tennessee", "Washington"]

# Note that 2022-11-08 is the 2022 regular season ish (I just estimated based on right before the playoffs typically are)
#year_list = ["2023-11-08", "2022-11-08", "2021-11-08", "2020-11-08", "2019-11-08", "2018-11-08", "2017-11-08", "2016-11-08",
#            "2015-11-08", "2014-11-08", "2013-11-08", "2012-11-08", "2011-11-08", "2010-11-08", "2009-11-08", "2008-11-08",
#            "2007-11-08", "2006-11-08", "2005-11-08", "2004-11-08"]

year_list = ["2022-11-08", "2021-11-08", "2020-11-08", "2019-11-08", "2018-11-08", "2017-11-08", "2016-11-08",
            "2015-11-08", "2014-11-08", "2013-11-08", "2012-11-08", "2011-11-08", "2010-11-08", "2009-11-08", "2008-11-08",
            "2007-11-08", "2006-11-08", "2005-11-08", "2004-11-08", "2003-11-08"]

#year_list = ["2023-11-08"]

link_list = ["https://www.teamrankings.com/nfl/stat/points-per-game",
            "https://www.teamrankings.com/nfl/stat/red-zone-scoring-pct",
            "https://www.teamrankings.com/nfl/stat/yards-per-game",
            "https://www.teamrankings.com/nfl/stat/yards-per-play",
            "https://www.teamrankings.com/nfl/stat/average-time-of-possession-net-of-ot",
            "https://www.teamrankings.com/nfl/stat/third-down-conversion-pct",
            "https://www.teamrankings.com/nfl/stat/yards-per-rush-attempt",
            "https://www.teamrankings.com/nfl/stat/rushing-attempts-per-game",
            "https://www.teamrankings.com/nfl/stat/pass-attempts-per-game",
            "https://www.teamrankings.com/nfl/stat/completions-per-game",
            "https://www.teamrankings.com/nfl/stat/yards-per-completion",
            "https://www.teamrankings.com/nfl/stat/average-team-passer-rating",
            "https://www.teamrankings.com/nfl/stat/kicking-points-per-game",
            "https://www.teamrankings.com/nfl/stat/net-punt-yards-per-game",
            "https://www.teamrankings.com/nfl/stat/field-goal-conversion-pct",
            "https://www.teamrankings.com/nfl/stat/opponent-points-per-game",
            "https://www.teamrankings.com/nfl/stat/opponent-red-zone-scoring-pct",
            "https://www.teamrankings.com/nfl/stat/opponent-yards-per-game",
            "https://www.teamrankings.com/nfl/stat/opponent-yards-per-play",
            "https://www.teamrankings.com/nfl/stat/opponent-average-time-of-possession-net-of-ot",
            "https://www.teamrankings.com/nfl/stat/opponent-third-down-conversion-pct",
            "https://www.teamrankings.com/nfl/stat/opponent-yards-per-rush-attempt",
            "https://www.teamrankings.com/nfl/stat/opponent-rushing-attempts-per-game",
            "https://www.teamrankings.com/nfl/stat/opponent-pass-attempts-per-game",
            "https://www.teamrankings.com/nfl/stat/opponent-completion-pct",
            "https://www.teamrankings.com/nfl/stat/opponent-yards-per-completion",
            "https://www.teamrankings.com/nfl/stat/opponent-average-team-passer-rating",
            "https://www.teamrankings.com/nfl/stat/opponent-kicking-points-per-game",
            "https://www.teamrankings.com/nfl/stat/opponent-net-punt-yards-per-game",
            "https://www.teamrankings.com/nfl/stat/opponent-field-goal-conversion-pct"]


#https://www.teamrankings.com/nfl/stat/points-per-game?date=2023-11-08

In [16]:
dataframe = pd.DataFrame()
first_column = True
headers = {'Accept-Encoding': 'identity'}
for link in link_list:
    first_year = True
    for year in year_list:
        #Blank data:
        stat_name = link[link.index('stat/')+5:]
        data_dictionary = {"Team":[], stat_name:[]}

        #Concat the link and year in a URL
        url = link + "?date=" + year
        contents = requests.get(url, headers=headers).text
        html = BeautifulSoup(contents, 'html.parser')

        for team in team_list:
            try: 
                value = html.find("td", {"data-sort":team}).find_next_sibling("td").text
            except:
                print(url)
                print(team)
                print(html)
                
                raise ValueError("hi")

            team_name = team + str(int(year[0:4]))
            data_dictionary['Team'].append(team_name)
            data_dictionary[stat_name].append(value)

        new_dataframe = pd.DataFrame(data_dictionary)
        if len(dataframe) == 0:
            dataframe = new_dataframe
        elif first_column == True:
            dataframe = pd.concat([dataframe, new_dataframe])
        elif first_year == True:
            new_dataframe.set_index('Team', inplace = True)
            dataframe = dataframe.merge(new_dataframe, left_index = True, right_index = True, how='left')
        else:
            new_dataframe.set_index('Team', inplace = True)
            dataframe.update(new_dataframe, overwrite = False)
        first_year = False

    if first_column == True:
        dataframe.set_index('Team', inplace = True)
        first_column = False #Now we have completed one column

dataframe.head(10)

KeyboardInterrupt: 

In [15]:
dataframe.to_csv("NFL_Team_Data.csv", index=True)

In [26]:
import pandas as pd

# Function to normalize team names to match the team_data index
def normalize_team_name_updated(name, year):
    # Mapping of full team names to city names as they appear in the team_data index
    team_name_mapping = {
        'Baltimore Ravens': 'Baltimore',
        'Dallas Cowboys': 'Dallas',
        'Indianapolis Colts': 'Indianapolis',
        'New England Patriots': 'New England',
        'Buffalo Bills': 'Buffalo',
        'Pittsburgh Steelers': 'Pittsburgh',
        'Tennessee Titans': 'Tennessee',
        'Tampa Bay Buccaneers': 'Tampa Bay',
        'Arizona Cardinals': 'Arizona',
        'Los Angeles Chargers': 'LA Chargers',
        'LA Chargers': 'LA Chargers',
        'Los Angeles Rams': 'LA Rams',
        'LA Rams': 'LA Rams',
        'St. Louis Rams': 'LA Rams',
        'Miami Dolphins': 'Miami',
        'Atlanta Falcons': 'Atlanta',
        'Jacksonville Jaguars': 'Jacksonville',
        'Cleveland Browns': 'Cleveland',
        'Detroit Lions': 'Detroit',
        'New Orleans Saints': 'New Orleans',
        'Washington Redskins': 'Washington',
        'Washington Football Team': 'Washington',
        'Washington Commanders': 'Washington',
        'Carolina Panthers': 'Carolina',
        'Minnesota Vikings': 'Minnesota',
        'Denver Broncos': 'Denver',
        'Green Bay Packers': 'Green Bay',
        'Kansas City Chiefs': 'Kansas City',
        'Oakland Raiders': 'Las Vegas',
        'Las Vegas Raiders': 'Las Vegas',
        'Philadelphia Eagles': 'Philadelphia',
        'Cincinnati Bengals': 'Cincinnati',
        'Houston Texans': 'Houston',
        'New York Giants': 'NY Giants',
        'New York Jets': 'NY Jets',
        'San Francisco 49ers': 'San Francisco',
        'Seattle Seahawks': 'Seattle',
        'Chicago Bears': 'Chicago'
    }
    normalized_name = team_name_mapping.get(name, None)
    if normalized_name:
        return normalized_name + str(year)
    else:
        return None  # If a name can't be normalized, we'll return None

# Function to extract year from the headings and map it correctly to each game
def extract_season_year(df):
    # Initialize a column for the season year
    df['Season_Year'] = None
    current_year = None

    # Loop through the dataframe to assign the correct season year
    for index, row in df.iterrows():
        if 'Regular Season' in str(row['Unnamed: 0']) or 'Playoffs' in str(row['Unnamed: 0']):
            current_year = str(row['Unnamed: 0']).split(' ')[0]
        df.at[index, 'Season_Year'] = current_year

    # Drop rows without a game date
    df = df[pd.to_datetime(df['Unnamed: 1'], errors='coerce', format='%b %d, %Y').notnull()]
    return df

# Function to add team stats to spread data
def add_team_stats(spread_data, team_data):
    favorite_team_stats = team_data.loc[spread_data['FavoriteTeamYear']].reset_index(drop=True)
    favorite_team_stats.columns = ['Favorite' + col for col in favorite_team_stats.columns]
    
    underdog_team_stats = team_data.loc[spread_data['UnderdogTeamYear']].reset_index(drop=True)
    underdog_team_stats.columns = ['Underdog' + col for col in underdog_team_stats.columns]
    
    final_data = pd.concat([spread_data.reset_index(drop=True), favorite_team_stats, underdog_team_stats], axis=1)
    return final_data

# Load the spreads data
spread_data_path = 'NFL_Spread_Data_2002-2022 - Adjusted Sheet.csv'
spread_data = pd.read_csv(spread_data_path)

# Extract season year from the headings
spread_data_with_year = extract_season_year(spread_data)

# Clean the data and rename the columns
spread_data_with_year = spread_data_with_year.rename(columns={
    'Unnamed: 0': 'Day', 'Unnamed: 1': 'Date', 'Unnamed: 2': 'Time', 'Unnamed: 4': 'Favorite',
    'Unnamed: 5': 'Score', 'Unnamed: 6': 'Spread', 'Unnamed: 8': 'Underdog', 'Unnamed: 9': 'OverUnder'
})

# Remove unnecessary columns and rows
spread_data_with_year = spread_data_with_year[['Day', 'Date', 'Time', 'Favorite', 'Score', 'Spread', 'Underdog', 'OverUnder', 'Season_Year']]
spread_data_with_year = spread_data_with_year.dropna(subset=['Day', 'Date'])

# Load the team data
team_data_path = 'NFL_Team_Data.csv'
team_data = pd.read_csv(team_data_path)
team_data.set_index('Team', inplace=True)

# Normalize team names and merge data
spread_data_with_year['FavoriteTeamYear'] = spread_data_with_year.apply(
    lambda row: normalize_team_name_updated(row['Favorite'], int(row['Season_Year'])), axis=1)
spread_data_with_year['UnderdogTeamYear'] = spread_data_with_year.apply(
    lambda row: normalize_team_name_updated(row['Underdog'], int(row['Season_Year'])), axis=1)
spread_data_with_year.dropna(subset=['FavoriteTeamYear', 'UnderdogTeamYear'], inplace=True)

# Add team stats to the cleaned spread data
final_dataset_all_years = add_team_stats(spread_data_with_year, team_data)

# Save the final dataset to a CSV file
final_dataset_path = 'Final_NFL_Spread_and_Team_Stats_2002-2021.csv'
final_dataset_all_years.to_csv(final_dataset_path, index=False)


In [58]:
import re
def favorite_covered(row):
    # Extracting numbers from the 'Score' column
    scores = re.findall(r'\d+', row['Score'])
    
    # Calculating the difference between the first and second numbers in 'Score'
    score_difference = int(scores[0]) - int(scores[1])
    
    print(row['Spread'])
    try:
        spread = re.search(r'(-?\d+(\.\d+)?)', row['Spread']).group(1)
    except:
        # Spread is 'pk'
        spread = 0
        
    # Comparing with the absolute value of 'Spread'
    if score_difference >= abs(float(spread)):
        return 'T'
    else:
        return 'F'

In [60]:
# Applying the function to create the 'Favorite Covered' column
final_dataset_all_years['Favorite Covered'] = final_dataset_all_years.apply(favorite_covered, axis=1)

# Displaying the result
print(final_dataset_all_years)

L -2.5
L -6.5
L -2.5
W -9
W -3.5
W -4.5
L -1
W -2.5
L -4
L -4
L -2.5
L -7.5
W -2.5
L -10.5
L -3
W -2.5
L -13
W -7.5
W -3
L -6.5
W -3.5
W -2.5
L -3
L -2.5
W -1.5
W -3.5
L -5
W -8
W -3.5
L -1
L -9.5
W -4.5
W -1
W -8
W -3
L -2
W -4
L -7
W -4.5
L -2.5
W -2.5
W -10.5
L -3
L -6
L -10
W -5.5
L -17
W -2
W -7
L -1.5
L -3.5
L -8.5
W -3.5
L -4
L -3
W -1.5
W -3
L -3.5
W -3
L -7
L -2.5
L -8.5
L -14.5
L -4
L -4
L -1
W -1.5
W -1.5
P -3
W -3.5
L -4.5
W -7
L -2
W -6.5
L -2.5
W -8
L -3
W -10
L -4
W -3
W -2
L -3
W -6.5
W -4.5
L -3.5
L -3
L -7.5
W -4
W -12.5
L -6
W -6
W -3
L -12.5
L -2.5
W -5.5
L -5.5
W -3.5
L -2
L -9.5
L -1.5
W -3
W -6.5
L -2.5
W -9
L -6.5
W -4
W -7.5
W -7
W -9
L -2.5
W -3.5
L -3.5
W -2.5
P -3
L -16
W -14
L -6
L -7
L -5
L -7.5
W -8
L -10
W -2.5
W -13
L -6
L -3
L -9.5
L -10.5
L -3
L -2
W -3.5
W -2.5
L -4
W -7
L -7
L -3.5
L -5
L -12.5
W -6.5
L -1
W -3.5
W -2.5
L -10
W -1
W -2.5
L -2.5
L -5.5
W -11
L -3
L -7.5
W -6.5
W -3.5
L -2.5
W -3
W -2
L -2.5
W -7
L -3.5
L -3
L -2
W -3.5
W -3.5
W PK
W 